In [1]:
from etl.dataloader import DataLoader, DataTransformer
from utils.os_ import get_git_root
import os

import pandas as pd

# Dataset preparation

## source info

In [2]:
git_root = get_git_root(os.getcwd())
path_to_file = os.path.join(git_root, "raw")
filename = "Analytics Interview Question_mobile_new_2024.xlsx"

## load raw dataset

In [3]:
data_loader = DataLoader()
df = data_loader.load_data_xlsx_from_tab(
    path=os.path.join(path_to_file, filename), 
    sheet_name="Data")

## Transform dataset

In [4]:
data_transformer = DataTransformer(df)
data_transformer.transform_data()
data_transformer.df.sample(10)

,Week,Mobile Indicator Name,Platform Type Name,Super Region,Country Name,Booking Window Group,Property Country,Net Gross Booking Value USD,Net Orders,Year,Date
64763,48,Desktop,Desktop,APAC,South Korea,8-14 days,Poland,798.0234,3,2023,2023-11-27
30782,48,Mobile,Mobile App,LATAM,Brazil,4-7 days,Germany,114.2394,1,2022,2022-11-28
31410,48,Mobile,Mobile App,North America,US,46-60 days,United Arab Emirates,295.9900,1,2022,2022-11-28
31867,48,Mobile,Mobile Web,APAC,Australia,0-1 days,Philippines,882.8998,3,2022,2022-11-28
22655,47,Mobile,Mobile App,North America,US,0-1 days,Germany,13563.4252,85,2022,2022-11-21
46414,46,Desktop,Desktop,EMEA,Norway,31-45 days,Russia,733.2564,5,2023,2023-11-13
69916,48,Mobile,Mobile App,EMEA,Norway,8-14 days,Sweden,1913.2256,8,2023,2023-11-27
54999,47,Desktop,Desktop,APAC,Australia,4-7 days,Russia,67.1421,1,2023,2023-11-20
20704,47,Desktop,Desktop,North America,US,4-7 days,Nigeria,1102.5500,1,2022,2022-11-21
34048,45,Desktop,Desktop,APAC,Australia,15-30 days,Fiji,-511.2341,2,2023,2023-11-06


In [ ]:
# Using restcountry API to get the region of the country in "Property Country" column

import requests
